In [1]:
#trying NER, followed by Custom NER

import spacy
#python -m spacy download en_core_web_sm
#downloaded english language support
#monkey learn could not be installed on anaconda


from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

###

In [2]:
doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
print([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'), ('Google', 'ORG'), ('$5.1 billion', 'MONEY'), ('Wednesday', 'DATE')]


In [3]:
print([(X, X.ent_iob_, X.ent_type_) for X in doc])

[(European, 'B', 'NORP'), (authorities, 'O', ''), (fined, 'O', ''), (Google, 'B', 'ORG'), (a, 'O', ''), (record, 'O', ''), ($, 'B', 'MONEY'), (5.1, 'I', 'MONEY'), (billion, 'I', 'MONEY'), (on, 'O', ''), (Wednesday, 'B', 'DATE'), (for, 'O', ''), (abusing, 'O', ''), (its, 'O', ''), (power, 'O', ''), (in, 'O', ''), (the, 'O', ''), (mobile, 'O', ''), (phone, 'O', ''), (market, 'O', ''), (and, 'O', ''), (ordered, 'O', ''), (the, 'O', ''), (company, 'O', ''), (to, 'O', ''), (alter, 'O', ''), (its, 'O', ''), (practices, 'O', '')]


In [4]:
from bs4 import BeautifulSoup
import requests
import re

In [5]:
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))


In [6]:
ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(ny_bb)
len(article.ents)

155

In [7]:
labels = [x.label_ for x in article.ents]
Counter(labels)

Counter({'ORG': 39,
         'PERSON': 77,
         'DATE': 23,
         'GPE': 9,
         'NORP': 2,
         'CARDINAL': 3,
         'LOC': 1,
         'ORDINAL': 1})

In [8]:
items = [x.text for x in article.ents]
Counter(items).most_common(3)

[('Strzok', 29), ('F.B.I.', 19), ('Trump', 13)]

In [9]:
sentences = [x for x in article.sents]
print(sentences[20])

A spokeswoman for the F.B.I. did not respond to a message seeking comment about why Mr. Strzok was dismissed rather than demoted.


In [10]:
displacy.render(nlp(str(sentences[20])), jupyter=True, style='ent')

In [11]:
displacy.render(nlp(str(sentences[20])), style='dep', jupyter = True, options = {'distance': 120})

In [12]:
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(str(sentences[20])) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]

[('spokeswoman', 'NOUN', 'spokeswoman'),
 ('F.B.I.', 'PROPN', 'F.B.I.'),
 ('respond', 'VERB', 'respond'),
 ('message', 'NOUN', 'message'),
 ('seeking', 'VERB', 'seek'),
 ('comment', 'NOUN', 'comment'),
 ('Mr.', 'PROPN', 'Mr.'),
 ('Strzok', 'PROPN', 'Strzok'),
 ('dismissed', 'VERB', 'dismiss'),
 ('demoted', 'VERB', 'demote')]

In [13]:
dict([(str(x), x.label_) for x in nlp(str(sentences[20])).ents])

{'F.B.I.': 'ORG', 'Strzok': 'PERSON'}

In [14]:
print([(x, x.ent_iob_, x.ent_type_) for x in sentences[20]])

[(A, 'O', ''), (spokeswoman, 'O', ''), (for, 'O', ''), (the, 'O', ''), (F.B.I., 'B', 'ORG'), (did, 'O', ''), (not, 'O', ''), (respond, 'O', ''), (to, 'O', ''), (a, 'O', ''), (message, 'O', ''), (seeking, 'O', ''), (comment, 'O', ''), (about, 'O', ''), (why, 'O', ''), (Mr., 'O', ''), (Strzok, 'B', 'PERSON'), (was, 'O', ''), (dismissed, 'O', ''), (rather, 'O', ''), (than, 'O', ''), (demoted, 'O', ''), (., 'O', '')]


In [15]:
l1=' Venezuela? Whos the president? This would be an unusual question to ask in most countries, but in Venezuela many want to know exactly that after opposition leader Juan Guaidó declared himself acting president on 23 January 2019. The move was a direct challenge to the power of President Maduro, who had been sworn in to a second six-year term in office just two weeks previously. Not surprisingly, President Maduro did not take '

displacy.render(nlp(l1), jupyter=True, style='ent')

In [17]:
l3=""" 
It wasn’t that long ago that Venezuela, which possesses the world’s largest crude oil reserves, was a relatively stable democracy with one of Latin America’s fastest-rising economies. 
It was a nation so awash in petroleum revenues that the socialist government of the late former President Hugo Chavez spent huge amounts on social programs and, at one point, even provided free heating oil for impoverished Americans. 
But starting in 2014, the South American nation began suffering a startling collapse. 
With Venezuela’s gross domestic product plummeting even more than the United States during the Great Depression, many of its nearly 32 million inhabitants became unable to afford food, and resource-starved hospitals did not have enough soap and antibiotics. 
Meanwhile, Venezuela’s political system spiraled into turmoil. President Nicolás Maduro, whose 2018 reelection was tainted by accusations of irregularities and voter intimidation, faced massive street protests and survived a spring 2019 military uprising instigated by opposition politician Juan Guaido, leader of the elected National Assembly whose legislative powers were taken away by Maduro’s regime in 2017. 
How is it that Venezuela sank so far in such a short time? Scholars who’ve studied the country say its rise and fall was caused by a combination of factors. “Venezuela has long been dependent on oil revenues, and the Bolivarian revolution of Hugo Chavez did not fundamentally alter that situation,” explains Jo-Marie Burt, an associate professor of political science and Latin American Studies at the Schar School of Policy and Government at George Mason University. 
“The decline of oil prices, the massive social spending of the Chavez and Maduro governments, U.S. sanctions, and a combination of economic mismanagement and corruption at the top have contributed to the economic collapse.” 
Venezuela's crisis has been deepened by U.S. sanctions against the Venezuelan oil industry. In March, it also sanctioned the Venezuelan gold mining industry, and in April, it also imposed sanctions against the Central Bank of Venezuela, cutting off that institution's access to U.S. currency and limiting its ability to conduct international transactions, to put even more pressure upon Maduro's regime. Here are some of the key moments in that saga. 
An undated photograph of the derricks in Maracaibo oil fields in Venezuela. An undated photograph of the derricks in Maracaibo oil fields in Venezuela. 1922: Oil Is Discovered An oil well in the Maracaibo basin of western Venezuela begins gushing 100,000 barrels of oil per day, indicating the massive reserves beneath the nation’s surface. 
Then-dictator Gen Juan Vicente Gomez allows more than 100 foreign oil companies into Venezuela and, by 1928, Venezuela becomes the world’s second-biggest petroleum exporter. 
The influx of oil revenues enriches Venezuela’s military regime, especially after it enacts a 1943 law requiring foreign oil companies to turn over half their profits. But the money only offers a band-aid on the nation’s underlying problems. 
“Even before the rise of the oil industry Venezuela did not have a highly productive agricultural sector,” says Miguel R. Tinker Salas, professor of Latin American studies and history at Pomona College in California, and author of Venezuela: What Everyone Needs to Know and The Enduring Legacy: Oil, Culture, and Society in Venezuela. 
“Land was monopolized by a handful of powerful families, infrastructure was lacking and the country lacked a nationally integrated economy.” But, Salas explains, oil and the rise of cities such as Caracas enabled people to flee rural poverty. 
Venezuelan President Romulo Betancourt, circa 1958. 1958: Venezuela Elects President Betancourt After the overthrow of brutal, corrupt Venezuelan dictator Marcos Pérez Jiménez, the nation’s three political parties agree to the Punto Fijo Pact to accept the results of popular elections, and opposition leader Rómulo Betancourt—widely regarded as the father of Venezuelan democracy—is elected president. 
But, as anthropologist Iselin Åsedotter Strønen has written, the power-sharing agreement also helps establish a system in which each of the parties is guaranteed a slice of government ministries, jobs and contracts, and keeps oil revenues in the hands of the government. 1973: OPEC Embargo Brings Billions The OPEC embargo against the U.S. and other countries causes the price of oil to quadruple, and Venezuela becomes the beneficiary. 
As billions more flow into the state treasury, its per-capita GDP soars throughout the rest of the decade. 
Two years later, Venezuelan President Carlos Andrés Pérez signs a law nationalizing the oil industry, creating a state-owned oil company called owned Petroleos de Venezuela, S.A. (PDVSA), and compelling foreign companies to give it a 60 percent ownership share in oil projects. 
1989: IMF Bailout After oil prices plummet due to a glut in the late 1980s, President Perez’s government struggles under the weight of $33 billion in foreign debt. Ultimately, Venezuela is forced to accept an International Monetary Fund bailout and impose austerity measures that result in sharp rises in the prices of consumer goods and fares for public transportation. Protesters take to the streets for demonstrations that turn violent, leading to a nationwide curfew and suspension of civil liberties. 
Hugo Chavez Venezuelan President Hugo Chavez speaking to citizens in 2000 regarding his economic plan. 1998: Hugo Chávez Elected Firebrand populist leader Hugo Chávez, a former lieutenant colonel in the Venezuelan military who six years earlier led a failed coup attempt, is elected president, upending a political establishment that had controlled the nation for decades. 
Over the next decade and a half, Chavez embarks on a massive social spending binge. “To finance educational, health, food, and housing programs for a population of over 30 million, the government redirected oil profits to address pressing social inequality,” Tinker Salas explains. “Chávez era social programs, which significantly reduced poverty, nonetheless increased dependence on oil.”Though Chavez wanted to diversify the Venezuelan economy, his expensive strategy only increased the dependence upon exported oil. 
Chavez also strives to build Venezuelan influence, providing subsidized oil to Cuba in exchange for the services of Cuban doctors and teachers. He sells oil to other South American countries and China at below-market rates. At the same time, though, Chavez neglects to spend money maintaining oil facilities, and production declines. After Hugo Chávez dies of cancer, he is replaced by his hand-picked successor Nicolás Maduro, who won election by just 1.6 percentage points over opponent Henrique Capriles, who unsuccessfully sought a recount. Muduro soon begins consolidating his power, using authority given him by the National Assembly to rule by decree. 
But he isn’t able to halt the decline in oil prices that pull down the Venezuelan economy with it.As Venezuela’s situation worsens, the Maduro regime arrests opposition political leaders and shuts down news websites and detains journalists, leading to criticism from the U.N. and the Inter-American Commission on Human Rights. In 2017, Venezuela’s Supreme Court, filled with Maduro loyalists, take away the powers of the National Assembly, increasing Maduro’s control. The following year, he is reelected overwhelmingly in an election that a coalition of other western hemisphere nations says “lacked legitimacy.”In January, opposition leader Guaido, head of the National Assembly, invokes the Venezuelan constitution to declare himself interim president, setting up a power struggle that is still yet to be resolved.

"""

displacy.render(nlp(l3), jupyter=True, style='ent')